# Agile systems

## Overview

BioSTEAM is able to conservatively design agile biorefineries that can switch configurations, vary feedstocks, and even vary operating conditions throughout the year. Switching **operation modes** throughout the year is modeled using an **AgileSystem** object, which simulates each mode of operation and compiles new **agile designs** for each unit operation suitable for all operation modes. For example, an agile distillation column would have the largest diameter and number of operating stages required among all operation modes. Each operation mode runs for a defined number of operating hours, allowing the variable operating costs (e.g. material and utility costs) and sales to be summed across different operation modes. This **preliminary** agile design assumption leads to **conservative** estimates around utility costs as some units may be **over-designed**. For example, a distillation column with a higher number of stages will require a lower reflux ratio and less utilities to reach a target separation. The following UML diagram summarizes how BioSTEAM agile biorefinery models work:

![Agile System UML](Agile_System_UML.png "Agile System UML")

## Agile sugarcane and sweet sorghum biorefinery

An AgileSystem object is first created, then operation modes are added. We will need two operation modes, one for sugarcane and one for sorghum. As an example, we expose fermentation efficiency as an operation parameter. Finally, we add operation modes for both sugarcane and sorghum processing and simulate:

In [3]:
import biosteam as bst
from biorefineries import sugarcane as sc

# Create the agile system object and define operation parameters
agile_sys = bst.AgileSystem()

# Create sugarcane system
sc.load()
sugarcane_sys = sc.sugarcane_sys

# Create new soghum system; which is exactly the same as sugarcane but with a different feedstock
sc.U101.ins[0] = None # Remove stream without warnings
sc.U101.ins[0] = sorghum = bst.Stream(
    'Sorghum',
    Water=0.7,
    Glucose=0.0111,
    Sucrose=0.126,
    Ash=0.006,
    Cellulose=0.0668,
    Hemicellulose=0.0394,
    Lignin=0.0358,
    Solids=0.015,
    total_flow=333334,
    units='kg/hr',
)
sorghum_sys = bst.System.from_units('sorghum_sys', sugarcane_sys.units)

@agile_sys.operation_parameter
def set_fermentation_efficiency(X):
    sc.R301.X = X
    
# The name of the parameter defaults to the name used in the function (X)
sugarcane_mode = agile_sys.operation_mode(system=sugarcane_sys, operating_hours=24*200, X=0.90)

# Assume fermentation of sorghum is less efficient for demonstration purposes
sorghum_mode = agile_sys.operation_mode(system=sorghum_sys, operating_hours=24*60, X=0.85) 

# Simulate and get results
agile_sys.simulate()

print(agile_sys)

AgileSystem(operation_modes=[OperationMode(system=sugarcane_sys, operating_hours=4800, X=0.9), OperationMode(system=sorghum_sys, operating_hours=1440, X=0.85)], operation_parameters={'X': <function set_fermentation_efficiency at 0x000002C80F399820>}, lang_factor=None)


In [ ]:
# Recreate the TEA object using the agile system
agile_tea = SugarcaneTEA(
    system=agile_sys,
    IRR=0.15,
    duration=(2018, 2038),
    depreciation='MACRS7',
    income_tax=0.35,
    operating_days=200,
    lang_factor=3,
    construction_schedule=(0.4, 0.6),
    WC_over_FCI=0.05,
    labor_cost=2.5e6,
    fringe_benefits=0.4,
    property_tax=0.001,
    property_insurance=0.005,
    supplies=0.20,
    maintenance=0.01,
    administration=0.005,
)


agile_tea.IRR = agile_tea.solve_IRR()
print('Agile Sugarcane/Sorghum')
print(f'NPV: {agile_tea.NPV/1e6:.2f}')
print(f'IRR: {agile_tea.IRR:.0%}')

While the IRR is higher due to increased production, the capital cost increases slightly due to the different feedstock composition:

In [ ]:
from math import ceil
sugarcane_mode.simulate()
TCI_sugarcane = sugarcane_tea.TCI
print('Total capital cost [MMUSD]')
print(f'Agile Sugarcane/Sorghum: {ceil(agile_tea.TCI / 1e6)}')
print(f'Sugarcane only: {ceil(TCI_sugarcane / 1e6)}')

### References

<a id='References'></a>

1. Huang, H., Long, S., & Singh, V. (2016) "Techno-economic analysis of biodiesel and ethanol co-production from lipid-producing sugarcane" Biofuels, Bioproducts and Biorefining, 10(3), 299–315. https://doi.org/10.1002/bbb.1640